In [1]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations
import copy

import bo_methods_lib
# from bo_methods_lib.bo_methods_lib.bo_functions_generic import gen_theta_set, clean_1D_arrays
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.analyze_data import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pympler
import pickle

from pympler import asizeof
import gpflow
import tensorflow_probability as tfp

from matplotlib import pyplot as plt

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-11 12:30:35.344863: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-11 12:30:35.400570: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 12:30:35.400614: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 12:30:35.402272: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to 

In [2]:
#Set Parameters
cs_name_val = 1
ep0 = 1 #Set initial ep as an even mix between exploration and exploitation
ep_enum_val = 1
meth_name_val = 4
sep_fact = 1.0
gen_heat_map_data = True
normalize = True
noise_mean = 0
noise_std = None
kernel_enum_val = 1
lenscl = None #list([0.136113749, 221.573761, 830.968019, 1.67917241, 0.3, 0.2])
outputscl = None
retrain_GP = 5
reoptimize_obj = 5
bo_iter_tot = 5
bo_run_total = 1
runs_per_job_max = 1
bo_runs_in_job = 1
save_data = False
ei_tol = 1e-7
obj_tol = 1e-7
num_x_data = 5
gen_meth_theta = 1 
gen_meth_x = 2
gen_meth_theta_val = 2
num_val_pts = 20
num_theta_multiplier = 2 #How many simulation data points to generate is equal to num_theta_multiplier*number of parameters
seed = 1

In [3]:
#Define method, ep_enum classes, indecies to consider, and kernel
meth_name = Method_name_enum(meth_name_val)
method = GPBO_Methods(meth_name)
ep_enum = Ep_enum(ep_enum_val)
kernel = Kernel_enum(kernel_enum_val)

#Define Simulator Class (Export your Simulator Object Here)
simulator = simulator_helper_test_fxns(cs_name_val, noise_mean, noise_std, seed)

#Generate Exp Data (OR Add your own experimental data as a Data class object)
set_seed = 1 #Set set_seed to 1 for data generation
gen_meth_x = Gen_meth_enum(gen_meth_x)
exp_data = simulator.gen_exp_data(num_x_data, gen_meth_x, set_seed)

#Set simulator noise_std artifically as 5% of y_exp mean (So that noise will be set rather than trained)
simulator.noise_std = np.abs(np.mean(exp_data.y_vals))*0.05

#Create Exploration Bias Class
if ep_enum.value == 1:
    #Constant value stays constant
    ep_bias = Exploration_Bias(ep0, None, ep_enum, None, None, None, None, None, None, None)
elif ep_enum.value == 2:
    #For decay method, decay from mixed to full exploitation (alpha of 0.5) for this example
    ep_bias = Exploration_Bias(ep0, None, ep_enum, None, bo_iter_tot, None, 0.5, None, None, None)
elif ep_enum.value == 3:
    #Set ep multiplier to 1.5 as recommended in Boyle
    ep_bias = Exploration_Bias(ep0, None, ep_enum, None, None, 1.5, None, None, None, None)
elif ep_enum.value == 4:
    #Jasrasaria method will take care of itself
    ep_bias = Exploration_Bias(None, None, ep_enum, None, None, None, None, None, None, None)
else:
    raise Warning("Ep_enum value must be between 1 and 4!")
    
#Generate Sim (Training) Data (OR Add your own training data here as a Data class object)
num_theta_data = len(simulator.indeces_to_consider)*num_theta_multiplier
gen_meth_theta = Gen_meth_enum(gen_meth_theta)
#Note at present, training data is always the same between jobs since we set the data generation seed to 1
sim_data = simulator.gen_sim_data(num_theta_data, num_x_data, gen_meth_theta, gen_meth_x, sep_fact, set_seed, False)

#Gen sse_sim_data and sse_sim_val_data
sim_sse_data = simulator.sim_data_to_sse_sim_data(method, sim_data, exp_data, sep_fact, False)

#Generate validation data if applicable. This is only useful for small (<5 Params + State Points). Otherwise this takes up too much memory
if num_val_pts > 0:
    gen_meth_theta_val = Gen_meth_enum(gen_meth_theta_val) #input is an integer (1 or 2)
    val_data = simulator.gen_sim_data(num_val_pts, num_x_data, gen_meth_theta_val, gen_meth_x, sep_fact, set_seed, True)
    val_sse_data = simulator.sim_data_to_sse_sim_data(method, val_data, exp_data, sep_fact, True)        
#Set validation data to None if not generating it
else:
    val_data = None
    val_sse_data = None
    gen_meth_theta_val = gen_meth_theta_val #Value is None
                    
#Define cs_name and cs_params class
#Signac saves all BO_Results in different folders, so they can have the same name
cs_name = "BO_Results"
cs_params = CaseStudyParameters(cs_name, ep0, sep_fact, normalize, kernel, lenscl, outputscl,
                                retrain_GP, reoptimize_obj, gen_heat_map_data, bo_iter_tot,
                                bo_runs_in_job, True, None, seed, ei_tol, obj_tol)
#Initialize driver class
driver = GPBO_Driver(cs_params, method, simulator, exp_data, sim_data, sim_sse_data, val_data, val_sse_data, None, ep_bias, gen_meth_theta)
#Get results
restart_bo_results = driver.run_bo_restarts()

In [4]:
print(restart_bo_results[0].results_df)

   index  Best Error  Exploration Bias    Max EI  \
0      0   16.440658               1.0  6.608749   
1      1   16.440658               1.0  7.610763   
2      2    4.220447               1.0  8.532974   
3      3    4.220447               1.0  1.918967   
4      4    4.220447               1.0  0.003306   

                                 Theta Max EI    Min Obj  Min Obj Act  \
0  [1.4934874301736423, -0.29385181267358723]  16.444913    16.040800   
1   [1.1864236975343498, -0.6624373391564086]  16.356832    16.402687   
2  [-0.7368704928696906, -1.0086490570267193]  -1.447668    79.898468   
3                                  [2.0, 2.0]   3.982541     6.591086   
4   [1.5790038897122665, -1.4942257267870929]   3.966428     4.505685   

                                Theta Min Obj Min Obj Cum.  \
0   [1.6881566899033191, -0.4233185286445511]      16.0408   
1   [1.6605229541435012, -0.4048780819020276]      16.0408   
2  [-1.8163265434255436, -0.8688060409882931]      16.0408   
